In [2]:
import os            # linux shell
import re            #^[]?%
import pymatgen as mg  #materials project
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob          #random
import matplotlib.pyplot as plt
import torch         #machine learning
import torch.nn as nn
from torch.autograd import Variable

In [3]:
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=16)

patt_xrd = xrd.XRDCalculator('CuKa')

train_path='/home/mii/trainCSSO/O5/train/'

test_path='/home/mii/trainCSSO/O5/test/'

global sample_num, rmat_num, atoms_num
sample_num=1
rmat_num=28  #row nums of the matrix for the input of CNN 
atoms_num=24

global move_num,extend_num
extend_num=100

LR_D=0.001  #learning rate
LR_G=0.001

In [4]:
move_num=-115

In [5]:
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    return folder

In [6]:
def tomgStructure(folder):
    POSfile=folder+'/CONTCAR'
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS
    #print(POSfile)
    '''
    with open(POSfile,'r+') as pp:
        transfer=pp.readlines()
        if transfer[5] =='   Ca   Sn   S    O \n':
            pass
        else:
            transfer.insert(5,'Ca   Sn   S    O\n')
            pp.seek(0)# move the zhizhen
            pp.truncate()#read by the byte
            pp.writelines(transfer)
        pp.close()
    '''   

In [7]:
def get_xrdmat4(mgStructure):
    global rmat_num
    xrd_data4 =patt_xrd.get_pattern(mgStructure)
    i_column = rmat_num
    xxx=[]
    yyy=[]
    mat4=[]
    xrd_i=len(xrd_data4)
    for i in range(xrd_i):
        if xrd_data4.y[i]>1 and xrd_data4.y[i] < 20:
            xxx.append(xrd_data4.x[i])
            yyy.append(xrd_data4.y[i])
    mat4.append(np.asarray(xxx))
    mat4.append(np.asarray(yyy))
    mat4=np.asarray(mat4)
    
    xrd_x=[]
    xrd_y=[]
    xrd_mat4=[]
    xrow=len(mat4[0])
    
    if xrow < i_column:
        for i in mat4[0]:
            xrd_x.append(i)
        for j in mat4[1]:
            xrd_y.append(j)
        for i in range(0,i_column-xrow):
            xrd_x.append(0)
            xrd_y.append(0)
        xrd_x=np.asarray(xrd_x)
        xrd_y=np.asarray(xrd_y)
    if xrow > i_column:
        xrd_x=mat4[0][:i_column]
        xrd_y=mat4[1][:i_column]
    if xrow == i_column:
        xrd_x= mat4[0]
        xrd_y= mat4[1]
        
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=np.dot(1/100,xrd_y)
    xrd_mat4.append(xrd_x)
    xrd_mat4.append(xrd_y)
    xrd_mat4=np.array(xrd_mat4)
    return xrd_mat4

In [8]:
def get_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy=np.float64(float(energy_string))
    return energy

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy
def get_energy_per_atom(energy):
    energy_per_atom=energy/atoms_num
    return energy_per_atom

In [9]:
get_energy(train_path+'0000')

-112.06258552

In [10]:
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat4(Random_Structure)
    multimat3_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat3_RS = np.asarray(np.sqrt(np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat3_RS

In [11]:
class GcssNet(nn.Module):
    def __init__(self, input_size=(sample_num,28,28)):
        super(GcssNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output

class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()
        self.Dlstm=nn.LSTM(
            input_size=sample_num,
            hidden_size=32,
            num_layers=1,
            batch_first=True,
        )
        self.out=nn.Sequential(
            nn.Linear(32,10),
            nn.ReLU(),
            nn.Linear(10,1),
            nn.Sigmoid(),
        )
        #nn.Linear(32,1)
        #nn.Relu
        #nn.Linear
        #nn.Sigmoid
        
    def forward(self,x):
        D_out,(h_n,h_c)=self.Dlstm(x,None)
        out = self.out(D_out[:,-1,:]) #(batch,time step,input)   
        return out

In [12]:
G1=GcssNet()
D1=DNet()

mat_Gl=[]    
mat_Dl=[]
pre_dd=[]
pre_gg=[]

In [45]:
opt_D1=torch.optim.Adam(D1.parameters(),lr=0.001)
opt_G1=torch.optim.Adam(G1.parameters(),lr=0.0001)

In [59]:
file_path=train_path
tfset=[]
for step in range(1,3):       
 
    sample_path=[]
    for i in range(1,sample_num + 1):
        path_ = random_xxpsk(file_path)
        sample_path.append(path_)
        tfset.append(path_)
    E_Gibbs=[]
    for path1_ in sample_path:
        try:
            total_energy=get_energy(path1_)
            tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
        except:
            print(path1_)
            
        E_Gibbs.append(tt_energy)
        
    E_Gibbs=np.asarray(E_Gibbs,dtype=np.float64)       
    E_Gibbs=Variable(torch.from_numpy(E_Gibbs[np.newaxis,np.newaxis,:]),requires_grad=True)
    #print(tfactor.shape)
    
    g_in=[]
    for path2_ in sample_path:
        path2_=str(path2_)                
        
        try:
            tomgS=tomgStructure(path2_)
            #print(tomgS)
            gin=GANs_Gmat(tomgS)
            #print(gin)
        except:
            pass
        g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    Gout=G1(g_in)
    #print(Gout.shape)
    
    prob_Tfactor_mat0=D1(E_Gibbs)
    prob_G1_mat1=D1(Gout)
    
    D1_loss=-torch.mean(torch.log(prob_Tfactor_mat0)+torch.log(1.-prob_G1_mat1))
    G1_loss=torch.mean(torch.log(1.-prob_G1_mat1))
    
    
    if step%1==0:
        opt_D1.zero_grad()
        D1_loss.backward(retain_graph=True)
        opt_D1.step()
        
        opt_G1.zero_grad()
        G1_loss.backward()
        opt_G1.step()
    else:
        opt_D1.zero_grad()
        D1_loss.backward()
        opt_D1.step()
    


    if step%1==0:
        print(step)
        gg=G1_loss.data.numpy().mean()
        dd=D1_loss.data.numpy().mean()
        print(dd)
        print(gg)
        print(prob_Tfactor_mat0.data.numpy().mean())
        print(prob_G1_mat1.data.numpy().mean())
        mat_Gl.append(gg)
        mat_Dl.append(dd)
        pre_dd.append(prob_Tfactor_mat0.data.numpy().mean())
        pre_gg.append(prob_G1_mat1.data.numpy().mean())
    
    
    if step%2==0:
        print('----------------------------------test-------------------------')
        file_path_test=test_path
        E_Gibbs=[]
        E_Gmodel=[]
        for step_test in range(10):
            sample_path=[]
            for i in range(1,sample_num + 1):
                path_ = random_xxpsk(file_path_test)
                sample_path.append(path_)
            
            for path1_ in sample_path:
                try:
                    total_energy=get_energy(path1_)
                    tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
                except:
                    print(path1_)
                    
            tt_energy=float(tt_energy)
            print('DFT',tt_energy)
            E_Gibbs.append(tt_energy)
        
                   

        #print(tfactor.shape)
    
            g_in=[]
            for path2_ in sample_path:
                path2_=str(path2_)                
        
                try:
                    tomgS=tomgStructure(path2_)
            #print(tomgS)
                    gin=GANs_Gmat(tomgS)
            #print(gin)
                except:
                    pass
                g_in.append(gin)
       
            g_in=np.asarray(g_in)
            g_in=g_in[np.newaxis,:,:,:] 
            g_in=np.asarray(g_in,dtype=np.float64) 
            g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
            Gout=G1(g_in)
        #print(Gout.shape)
    
            G_data=Gout.data.numpy().mean()
            print('G_predict',G_data)
            E_Gmodel.append(G_data)

        
            
            
            print('error',abs(tt_energy-G_data))
            
        print('------------------end-test----------------------------')
        print(abs(np.asarray(E_Gibbs)-np.asarray(E_Gmodel)).mean())
    else:
        pass

1
1.3576454811462653
-0.6432546427905481
0.4894901996501493
0.47442093474553504
2
1.3311588218075676
-0.6387131179276754
0.5003508613861793
0.4720285759868392
----------------------------------test-------------------------
DFT 260.97625400000055
G_predict 254.74453448549446
error 6.231719514506096
DFT 289.50894400000067
G_predict 270.8354646566733
error 18.673479343327358
DFT 253.04373200000043
G_predict 254.1260414134763
error 1.0823094134758549
DFT 267.03516999999977
G_predict 273.57363104861895
error 6.538461048619183
DFT 287.57660200000004
G_predict 265.99324025611935
error 21.583361743880687
DFT 271.75214600000004
G_predict 258.2389896940479
error 13.513156305952123
DFT 231.86393200000026
G_predict 273.6859257469181
error 41.821993746917826
DFT 259.5076969999994
G_predict 276.0105590835289
error 16.50286208352952
DFT 253.043787
G_predict 254.1260414134763
error 1.082254413476278
DFT 274.71348800000044
G_predict 284.3439702717659
error 9.630482271765459
------------------end-test--

In [60]:
torch.save(G1.state_dict(),"/home/mii/trainCSSO/G1_o5_2.pkl") 
torch.save(D1.state_dict(),"/home/mii/trainCSSO/D1_o5_2.pkl")

In [64]:
E_Gibbs_test=[]
E_Gmodel_test=[]
abserrset=[]
err0set=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)
        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        GGG=get_energy_per_atom(GGG)
        E_Gibbs_test.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        G_data=get_energy_per_atom(G_data)
        E_Gmodel_test.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        abserrset.append(abserr)
        err0=abs(abserr/GGG)
        err0set.append(err0)
    except:
        print(path_)

In [65]:
print(np.asarray(err0set).mean())

0.0019507858791029222


In [66]:
print(np.asarray(abserrset).mean())

0.009138452379169826


In [61]:
E_Gibbs_t=[]
E_Gmodel_t=[]
abs_t_errset=[]
err_t_0set=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)
#'''        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        GGG=get_energy_per_atom(GGG)
        E_Gibbs_t.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        G_data=get_energy_per_atom(G_data)
        E_Gmodel_t.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        abs_t_errset.append(abserr)
        err0=abs(abserr/GGG)
        err_t_0set.append(err0)
    except:
        print(path_)

In [62]:
print(np.asarray(abs_t_errset).mean())

0.006024828361763324


In [63]:
print(np.asarray(err_t_0set).mean())

0.0012885441669967035


In [67]:
print(mat_Dl)

[0.9824571696235957, 0.3245490596005498, 0.05271953471401261, 0.010620221098517225, 0.003967261521465131, 0.002297823956658886, 0.001715235597395944, 0.0015029799064919668, 0.04487686769822716, 3.7320203083544707, 4.2306324726394084, 3.832249029853866, 2.698926990497396, 1.8212192888656349, 1.3746364660691388, 1.2004729145788962, 1.1336753989822603, 1.094590838942578, 1.1068600643767028, 1.1409449586495404, 1.1131542201504487, 1.1267068415910404, 1.156800833380355, 1.1569063562333817, 1.189716077696089, 1.2087121058662489, 1.2126253955409427, 1.240061789499331, 1.301281037413915, 1.304356891087307, 1.3123537484111396, 1.316101323313283, 1.3904398400721787, 1.4044655589085204, 1.4624115906058686, 1.4908819131858995, 1.487451871012247, 1.4958936738866873, 1.5082638796488785, 1.479660226522748, 1.4502640253513988, 1.4262681372742763, 1.3895633969954453, 1.359883261279094, 1.3480552964220762, 1.330019348488816, 1.3283333873680117, 1.3218716498435437, 1.310093070725623, 1.3035227564790444, 

In [68]:
print(pre_dd)

[0.6468913765324028, 0.8687875773695217, 0.9775461455848394, 0.9952762445157707, 0.9981586524266647, 0.998912373325715, 0.9991916041243508, 0.9993257193932931, 0.9993651074522314, 0.9989431306658646, 0.9971115707253605, 0.979308071062282, 0.8969816462504129, 0.7817007194815964, 0.6273901075251628, 0.6042207603940427, 0.6396340789532043, 0.6334756944739626, 0.6252194520098311, 0.59829101496908, 0.5987997743806401, 0.5837434537830354, 0.5709505900920748, 0.5846307459252623, 0.5674420735260137, 0.5581033075077695, 0.566951734294719, 0.5477887086669702, 0.5237468343444303, 0.5236228167494806, 0.5278704448017844, 0.5281668779468844, 0.499031856669804, 0.49392053848988543, 0.46613802622433065, 0.45577352917833386, 0.45802693194397104, 0.44625362247499606, 0.4229021421795846, 0.417936358911013, 0.4195852305309468, 0.43436656253865213, 0.46264971205952443, 0.4959090830907128, 0.51885006882661, 0.5214034434997432, 0.511680668553722, 0.5139545776494954, 0.5280715225479033, 0.5441015143795457, 0.

In [69]:
print(mat_Gl)

[-0.5468802831533321, -0.18389243108803277, -0.030009754236571717, -0.005885273421319, -0.0021242165837416643, -0.0012096053872697517, -0.0009065127935974262, -0.0008284718703768361, -0.044241773520838336, -3.7309628801401304, -4.2277398638027455, -3.8113400232292576, -2.590207112105426, -1.5749359655156483, -0.9084497153063984, -0.6966572640948222, -0.6868163812059277, -0.6380571919358178, -0.6372074967173667, -0.6272669623492202, -0.6003262169278758, -0.5884131574529615, -0.5963482280720506, -0.6201315216437152, -0.6230994697777837, -0.6255009110321437, -0.6451442919625728, -0.6381961549887408, -0.6545341854387474, -0.657373221871159, -0.6734493533336052, -0.6777583348195886, -0.6953544958254602, -0.6990849309166915, -0.6991380955071521, -0.7051226738096656, -0.7066245777865603, -0.6890258455564792, -0.6476494105841983, -0.6072341170648852, -0.5817754233466136, -0.592401650051092, -0.6187783243360029, -0.6585205920000089, -0.6919149741434487, -0.6787881753182633, -0.6582788446162147,

In [70]:
print(pre_gg)

[0.421247459034008, 0.16797470093071745, 0.029563932360639568, 0.005867989123925652, 0.0021219620323620647, 0.001208874109555507, 0.0009061020350036144, 0.0008281287823096927, 0.04327738068768406, 0.9760302552696555, 0.9854146818705567, 0.9778814802408057, 0.9249954958642271, 0.792979191272537, 0.5968512648631169, 0.5017519651965593, 0.49682455939001535, 0.47168215220510995, 0.47123305228400875, 0.46595061463927995, 0.4513673663534809, 0.44479238685853356, 0.4491805651993156, 0.4621263090895234, 0.46372032366043825, 0.4650066226951259, 0.47541315701977704, 0.4717555637652879, 0.4803159173726185, 0.48178922703704663, 0.4900534452350371, 0.4922460681200583, 0.501102440468307, 0.5029600783782279, 0.5029865025996808, 0.5059520362073165, 0.5066934918719317, 0.4979350803081544, 0.4767256646087856, 0.4551442057649404, 0.4410948081526537, 0.4470024180376967, 0.46139796717507664, 0.4823834656273978, 0.4993835170526321, 0.4927687045202129, 0.48225831805796526, 0.4812074891041566, 0.4890939320054

In [71]:
print(err0set)

[0.002625553378328337, 0.0015249292365289598, 0.001850455280914732, 0.0007874909587982029, 0.0008208067525291724, 0.003675379856442459, 0.0017631880534687697, 0.000387644959756354, 0.004897212838538805, 0.003124623897205708, 0.001082110707871699, 0.005850418304698827, 0.0035307289359356436, 0.0010484879975795219, 0.0011614899265143099, 0.0007461570395187184, 0.0025968911760623696, 7.783609678778753e-05, 0.0006942202180746057, 0.0008581471968275831, 0.0008227204161794373, 0.0011244131499440313, 0.001995219511898677, 0.0019116576304306205, 0.0004331050068095686, 0.00024001012315560247, 0.0008714319461769113, 0.004893950879246736, 0.0053184686034069405, 0.004652167289109333, 0.003030710155228811, 0.00017203855213128945, 0.0022390880014866426, 0.0027633992016449396, 0.0025433701381300296, 0.001929935396854388, 0.0010067427718179398, 0.002389645191674409, 0.0018051327059987292, 0.0005341891327863568, 0.00017261586725163762, 0.005704696431412911, 0.00027829382582619496, 0.0007365489743544511

In [72]:
print(abserrset)

[0.012277083844709935, 0.00714352301407839, 0.008647545030446402, 0.0036827064861215675, 0.0038444960320873633, 0.01726805779993601, 0.00822666187687382, 0.0018135065274869078, 0.02303125860392452, 0.014554124676362967, 0.005061629479605756, 0.027528018706849622, 0.01656417037499125, 0.004918228248369516, 0.0054398908223145526, 0.003496884346200879, 0.012169465030694049, 0.0003644162348965807, 0.0032460956303967947, 0.0040137292799027335, 0.0038548251688172996, 0.005265500243480048, 0.009324651186116029, 0.00892343567053544, 0.0020271929361603824, 0.0011217190998085158, 0.0040720180869113776, 0.02301541839558663, 0.02500457474630835, 0.021675836383979075, 0.014242142598837937, 0.000805077314355529, 0.01046764641707032, 0.012963042079463527, 0.011923905421518555, 0.009052250900709602, 0.004719769862547629, 0.011156731085221416, 0.008431247924406193, 0.0025001862473592595, 0.0008089216230944274, 0.026854307346300565, 0.001302577647825487, 0.0034533700812877655, 0.018911180751878476, 0.01

In [73]:
print(abs_t_errset)

[0.0015996757125860128, 0.01480958919369968, 0.010913178003491453, 0.010129778610419748, 0.009406201457419883, 0.0015635981150001754, 0.00994293051290196, 0.0034066863693089644, 0.0052828486130387375, 0.002728292594993853, 0.011878071942698298, 0.0022345080410177687, 0.006017540810036692, 0.0051015045182749574, 0.004959174662023891, 0.001566436733737575, 0.010147080639604233, 0.009259391971234265, 8.851965911471638e-05, 0.0026515983453183622, 0.00918458875117345, 0.0048285161640286844, 0.0010139500453050232, 0.0011049956672861683, 0.010915339670158453, 0.008142010060358196, 0.0017678450266727097, 0.008348693426930787, 0.0008942001183758208, 0.0014970461836476545, 0.005963452186879437, 0.01422511056915532, 0.01067848138669003, 0.0029433441979591635, 0.010170338387270661, 0.005537170016690851, 0.0059634209368795155, 0.008028562393505467, 0.0014751766688441137, 0.0031310214122912683, 0.002728830094993917, 0.005387364044053378, 0.008345894273518617, 0.005178638951223391, 0.0070258800236056

In [74]:
print(err_t_0set)

[0.0003419631438342827, 0.0031707767336027174, 0.002336911325320691, 0.002167169225715418, 0.0020124383981188275, 0.00033401708504402313, 0.002125843096631631, 0.0007291299667155972, 0.0011295554346715292, 0.0005824271795572105, 0.0025431011718639635, 0.00047736996442031575, 0.0012856196839373976, 0.0010905338696045512, 0.0010601070132319583, 0.0003343751001164211, 0.002168906774946447, 0.001981154750916721, 1.8905623507369927e-05, 0.0005666293930964464, 0.001965075971446486, 0.001030237769793709, 0.00021661064092927726, 0.00023602020274134133, 0.0023373752993210806, 0.0017429509139303403, 0.00037719413427102726, 0.0017867830516856912, 0.00019124598937092044, 0.0003198300331743358, 0.0012769470140594577, 0.003044109134295864, 0.002284248400315627, 0.000629054078399758, 0.002167700221024958, 0.0011839906333537752, 0.0012769403139889916, 0.0017178112821315782, 0.00031496923845827457, 0.0006696357741851609, 0.000582541856495531, 0.0011520358945708252, 0.00178561204479487, 0.00110722623710